# Spectrogram_ResNet50

In [17]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
from keras import layers, metrics, applications
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras import layers
import glob
import os
from PIL import Image
from random import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import h5py

In [18]:
batch_size = 64
epochs = 150

In [19]:
infile = open('./csDataframe.pkl', 'rb')
data = pickle.load(infile)
infile.close()
set = []
for row in data.itertuples():
    #x=loaded[row.File_Name]['x'][int(row.Start*loaded[row.File_Name]['sr']):int(row.End*loaded[row.File_Name]['sr'])]
    x, sr = row.npArr , 4000
    
    X = np.abs(librosa.stft(x,n_fft = 511, hop_length = 128))
    Xdb = librosa.amplitude_to_db(abs(X))

    np.reshape(Xdb,(1,256,94,1))

    
    if(row[5] == 0 and row[6] == 0): set.append((Xdb,0))
    else: set.append((Xdb,1))

x,y = zip(*set)

from sklearn.model_selection import train_test_split
combine_X, test_X, combine_y, test_y = train_test_split(
    x,
    y,
    test_size = 0.2, 
    random_state = 2018)
train_X, valid_X, train_y, valid_y = train_test_split(
    combine_X,
    combine_y,
    test_size = 0.25, 
    random_state = 2018)
print('Training', np.shape(train_X), np.shape(train_y))
print('Validation', np.shape(valid_X), np.shape(valid_y))
print('Test', np.shape(test_X), np.shape(test_y))


Training (4138, 256, 94) (4138,)
Validation (1380, 256, 94) (1380,)
Test (1380, 256, 94) (1380,)


In [20]:
train_X=np.array(train_X)
train_y = np.array(train_y)
valid_X= np.array(valid_X)
valid_y= np.array(valid_y)
test_X=np.array(test_X)
test_y = np.array(test_y)

In [21]:
import tensorflow as tf
from keras.models import *
from keras.layers import *

In [22]:
def top_3_accuracy(x, y): 
    return metrics.sparse_top_k_categorical_accuracy(x,y,3)
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [23]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [24]:
def ResNet50(input_shape=(256, 94, 1) ,classes = train_X):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

model = ResNet50()
opt = tf.optimizers.Adam(lr=1e-3)
model.compile(optimizer=opt, 
    loss=losses.binary_crossentropy, 
    metrics=['accuracy', 'Precision', 'AUC', tf.keras.metrics.Recall(), tf.keras.metrics.TrueNegatives()])

C:\Users\Rafia.Alice\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [25]:
base_model = ResNet50(input_shape=(256, 94, 1))

In [26]:
headModel = base_model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( 1,activation='sigmoid', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [27]:
model = Model(inputs=base_model.input, outputs=headModel)

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 94, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 zero_padding2d_3 (ZeroPadding2  (None, 262, 100, 1)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 128, 47, 64)  3200        ['zero_padding2d_3[0][0]']       
                                                                                            

In [29]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
# weight_path="{}_weights.best.hdf5".format('spectro_sound_model')
checkpoint = ModelCheckpoint(filepath ="./check.hDf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
# reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', 
#                                    factor=0.8, patience=5, 
#                                    verbose=1, mode='auto', 
#                                    epsilon=0.0001, cooldown=5, 
#                                    min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early]

In [30]:
opt = tf.optimizers.Adam(lr=1e-3)
model.compile(optimizer=opt, 
    loss=losses.binary_crossentropy, 
    metrics=['accuracy', 'Precision', 'AUC', tf.keras.metrics.Recall(), tf.keras.metrics.TrueNegatives()])

In [31]:
from IPython.display import clear_output
train_X=np.array(train_X)
train_y = np.array(train_y)
fit_results = model.fit(train_X, train_y,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(valid_X,valid_y), 
          callbacks=callbacks_list)
# clear_output()

Epoch 1/150
65/65 [==============================] - ETA: 0s - loss: 1.5026 - accuracy: 0.5162 - precision: 0.4835 - auc: 0.5201 - recall_3: 0.4590 - true_negatives_3: 1245.0000 
Epoch 1: val_loss improved from inf to 4.84562, saving model to .\check.hDf5
65/65 [==============================] - 743s 11s/step - loss: 1.5026 - accuracy: 0.5162 - precision: 0.4835 - auc: 0.5201 - recall_3: 0.4590 - true_negatives_3: 1245.0000 - val_loss: 4.8456 - val_accuracy: 0.4797 - val_precision: 0.4797 - val_auc: 0.4467 - val_recall_3: 1.0000 - val_true_negatives_3: 0.0000e+00
Epoch 2/150
65/65 [==============================] - ETA: 0s - loss: 0.7177 - accuracy: 0.5536 - precision: 0.5287 - auc: 0.5764 - recall_3: 0.4467 - true_negatives_3: 1424.0000
Epoch 2: val_loss improved from 4.84562 to 0.68125, saving model to .\check.hDf5
65/65 [==============================] - 678s 10s/step - loss: 0.7177 - accuracy: 0.5536 - precision: 0.5287 - auc: 0.5764 - recall_3: 0.4467 - true_negatives_3: 1424.0000

In [32]:
model.load_weights("./check.hDf5")
model.evaluate(x=test_X, y=test_y)

44/44 [==============================] - 47s 1s/step - loss: 0.6305 - accuracy: 0.6558 - precision: 0.6236 - auc: 0.7168 - recall_3: 0.6876 - true_negatives_3: 456.0000


[0.6305264830589294,
 0.6557971239089966,
 0.6236110925674438,
 0.7168280482292175,
 0.6875957250595093,
 456.0]